In [0]:

# import pyspark functions
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, TimestampType, FloatType
# import URL processing
import urllib

In [0]:
# Define the path to the Delta table
delta_table_path = "dbfs:/user/hive/warehouse/authentication_credentials"

# Read the Delta table to a Spark DataFrame
aws_keys_df = spark.read.format("delta").load(delta_table_path)

In [0]:
# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.select('Secret access key').collect()[0]['Secret access key']

# Encode the secret key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

In [0]:
def get_kinesis_stream(stream_name: str):
    '''Uses spark.readStream to retrieve Kinesis stream and returns stream as dataframe'''
    dataframe = spark.readStream \
    .format('kinesis') \
    .option('streamName', stream_name) \
    .option('initialPosition','earliest') \
    .option('region','us-east-1') \
    .option('awsAccessKey', ACCESS_KEY) \
    .option('awsSecretKey', SECRET_KEY) \
    .load()
     # .option('initialPosition','earliest')  check earliest vs latest
    return dataframe

def deserialize_kinesis_stream(stream, schema):
    '''Takes stream dataframe and schema, deserializes data from stream and returns data as dataframe'''
    dataframe = stream \
    .selectExpr("CAST(data as STRING)") \
    .withColumn("data", from_json(col("data"), schema)) \
    .select(col("data.*"))
    return dataframe

def write_Kinesis_stream_df_to_table(dataframe, prefix: str, name: str):
    '''Takes dataframe and name string and writes dataframe to delta table using name in options and table name'''
    dataframe.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", f"/tmp/kinesis/{prefix}_{name}_table_checkpoints/") \
    .table(f"{prefix}_{name}_table")

In [0]:
# define schemas for each of the dataframes
pin_schema = StructType([
    StructField("index", IntegerType()),
    StructField("unique_id", StringType()),
    StructField("title", StringType()),
    StructField("description", StringType()),
    StructField("poster_name", StringType()),
    StructField("follower_count", StringType()),
    StructField("tag_list", StringType()),
    StructField("is_image_or_video", StringType()),
    StructField("image_src", StringType()),
    StructField("downloaded", IntegerType()),
    StructField("save_location", StringType()),
    StructField("category", StringType())
])
geo_schema = StructType([
    StructField("ind", IntegerType()),
    StructField("timestamp", TimestampType()),
    StructField("latitude", FloatType()),
    StructField("longitude", FloatType()),
    StructField("country", StringType())
])
user_schema = StructType([
    StructField("ind", IntegerType()),
    StructField("first_name", StringType()),
    StructField("last_name", StringType()),
    StructField("age", StringType()),
    StructField("date_joined", TimestampType())
])

In [0]:
kstream_pin = get_kinesis_stream('streaming-0affec486183-pin')
# kstream_geo = get_kinesis_stream('streaming-0affec486183-geo')
# kstream_user = get_kinesis_stream('streaming-0affec486183-user')
display(kstream_pin)

partitionKey,data,stream,shardId,sequenceNumber,approximateArrivalTimestamp
partition-1,eyJpbmRleCI6NzUyOCwidW5pcXVlX2lkIjoiZmJlNTNjNjYtMzQ0Mi00NzczLWIxOWUtZDNlYzZmNTRkZGRmIiwidGl0bGUiOiJObyBUaXRsZSBEYXRhIEF2YWlsYWJsZSIsImRlc2NyaXB0aW9uIjoiTm8gZGVzY3JpcHRpb24= (truncated),streaming-0affec486183-pin,shardId-000000000000,49654776698500171106604173156775567464098671253650407426,2024-08-12T12:21:11.473+0000
partition-1,eyJpbmRleCI6Mjg2MywidW5pcXVlX2lkIjoiOWJmMzk0MzctNDJhNi00ZjAyLTk5YTAtOWEwMzgzZDhjZDcwIiwidGl0bGUiOiIyNSBTdXBlciBGdW4gU3VtbWVyIENyYWZ0cyBmb3IgS2lkcyAtIE9mIExpZmUgYW5kIExpc2E= (truncated),streaming-0affec486183-pin,shardId-000000000000,49654776698500171106604173161714029437224431569764089858,2024-08-12T12:21:13.127+0000
partition-1,eyJpbmRleCI6NTczMCwidW5pcXVlX2lkIjoiMWUxZjBjOGItOWZjZi00NjBiLTkxNTQtYzc3NTgyNzIwNmViIiwidGl0bGUiOiJJc2xhbmQgT2FzaXMgQ291cG9uIE9yZ2FuaXplciIsImRlc2NyaXB0aW9uIjoiRGVzY3JpcHQ= (truncated),streaming-0affec486183-pin,shardId-000000000000,49654776698500171106604173170971983363833261927102939138,2024-08-12T12:21:16.233+0000
partition-1,eyJpbmRleCI6ODMwNCwidW5pcXVlX2lkIjoiNWI2ZDA5MTMtMjVlNC00M2FiLTgzOWQtODVkNTUxNmY3OGE0IiwidGl0bGUiOiJUaGUgIzEgUmVhc29uIFlvdeKAmXJlIE5vdCBIaXMgUHJpb3JpdHkgQW55bW9yZSAtIE1hdHQ= (truncated),streaming-0affec486183-pin,shardId-000000000000,49654776698500171106604173179545685276540212240277569538,2024-08-12T12:21:18.745+0000
partition-1,eyJpbmRleCI6ODczMSwidW5pcXVlX2lkIjoiZWE3NjBmNzEtZmViZi00MDIzLWI1OTItZDE3Mzk2NjU5MDM5IiwidGl0bGUiOiIyMCBLb2kgRmlzaCBUYXR0b29zIEZvciBMdWNreSBNZW4iLCJkZXNjcmlwdGlvbiI6IktvaSA= (truncated),streaming-0affec486183-pin,shardId-000000000000,49654776698500171106604173183333249869392845513351495682,2024-08-12T12:21:19.926+0000
partition-1,eyJpbmRleCI6MTMxMywidW5pcXVlX2lkIjoiNDQ2NjIwNDUtZTg5MS00ODIxLThhMTktZWJlN2VlZGQzNzFhIiwidGl0bGUiOiJMaXF1aWQgTGFzaCBFeHRlbnNpb25zIE1hc2NhcmEiLCJkZXNjcmlwdGlvbiI6Ikluc3RhbnQ= (truncated),streaming-0affec486183-pin,shardId-000000000000,49654776698500171106604173185143011821355945456606117890,2024-08-12T12:21:20.543+0000
partition-1,eyJpbmRleCI6NDMxNSwidW5pcXVlX2lkIjoiMjFiNTliYTktODI5ZC00YzMzLThjMjctNGNkNGM1NmQyNmI4IiwidGl0bGUiOiJQb2RjYXN0cyBmb3IgVGVhY2hlcnMgb3IgUGFyZW50cyBvZiBUZWVuYWdlcnMiLCJkZXNjcmk= (truncated),streaming-0affec486183-pin,shardId-000000000000,49654776698500171106604173192035097918978946587764457474,2024-08-12T12:21:23.197+0000
partition-1,eyJpbmRleCI6MTA3OTQsInVuaXF1ZV9pZCI6ImM0YmQyNTc3LWE3YmItNDQwOS1iYjdhLTE3ZDVlZDdlMWNmMSIsInRpdGxlIjoiVGlyZUJ1eWVyIiwiZGVzY3JpcHRpb24iOiJOaXNzYW4gR1QtUi4gU2ljay4iLCJwb3N0ZXI= (truncated),streaming-0affec486183-pin,shardId-000000000000,49654776698500171106604173194183359100434142699936808962,2024-08-12T12:21:23.874+0000
partition-1,eyJpbmRleCI6NTQ5NCwidW5pcXVlX2lkIjoiOGZiMmFmNjgtNTQzYi00NjM5LTgxMTktZGUzM2QyODcwNmVkIiwidGl0bGUiOiJEYXZlIFJhbXNleSdzIDcgQmFieSBTdGVwczogV2hhdCBBcmUgVGhleSBBbmQgV2lsbCBUaGU= (truncated),streaming-0affec486183-pin,shardId-000000000000,49654776698500171106604173195962897906906876845104300034,2024-08-12T12:21:24.503+0000
partition-1,eyJpbmRleCI6NTA2OSwidW5pcXVlX2lkIjoiYjc1YjZmODctZGViMy00NDRmLWIyOWUtY2U5MTYxYjJkZjQ5IiwidGl0bGUiOiJUaGUgVmF1bHQ6IEN1cmF0ZWQgJiBSZWZpbmVkIFdlZGRpbmcgSW5zcGlyYXRpb24iLCJkZXM= (truncated),streaming-0affec486183-pin,shardId-000000000000,49654776698500171106604173199009390972335742434083340290,2024-08-12T12:21:25.730+0000


In [0]:
# need to be sending live data for this to work:
kstream_geo = get_kinesis_stream('streaming-0affec486183-geo')


In [0]:
display(kstream_geo)

partitionKey,data,stream,shardId,sequenceNumber,approximateArrivalTimestamp
partition-geo,eyJpbmQiOjc1MjgsInRpbWVzdGFtcCI6IjIwMjAtMDgtMjhUMDM6NTI6NDciLCJsYXRpdHVkZSI6LTg5Ljk3ODcsImxvbmdpdHVkZSI6LTE3My4yOTMsImNvdW50cnkiOiJBbGJhbmlhIn0=,streaming-0affec486183-geo,shardId-000000000000,49654776715983955342252074455743937999881516150966714370,2024-08-12T17:29:20.362+0000
partition-geo,eyJpbmQiOjI4NjMsInRpbWVzdGFtcCI6IjIwMjAtMDQtMjdUMTM6MzQ6MTYiLCJsYXRpdHVkZSI6LTUuMzQ0NDUsImxvbmdpdHVkZSI6LTE3Ny45MjQsImNvdW50cnkiOiJBcm1lbmlhIn0=,streaming-0affec486183-geo,shardId-000000000000,49654776715983955342252074458108596903047730954130948098,2024-08-12T17:29:21.974+0000
partition-geo,eyJpbmQiOjU3MzAsInRpbWVzdGFtcCI6IjIwMjEtMDQtMTlUMTc6Mzc6MDMiLCJsYXRpdHVkZSI6LTc3LjAxNSwibG9uZ2l0dWRlIjotMTAxLjQzNywiY291bnRyeSI6IkNvbG9tYmlhIn0=,streaming-0affec486183-geo,shardId-000000000000,49654776715983955342252074460300379414009053716592721922,2024-08-12T17:29:23.602+0000
partition-geo,eyJpbmQiOjgzMDQsInRpbWVzdGFtcCI6IjIwMTktMDktMTNUMDQ6NTA6MjkiLCJsYXRpdHVkZSI6LTI4Ljg4NTIsImxvbmdpdHVkZSI6LTE2NC44NywiY291bnRyeSI6IkZyZW5jaCBHdWlhbmEifQ==,streaming-0affec486183-geo,shardId-000000000000,49654776715983955342252074462366433639730455113604530178,2024-08-12T17:29:25.236+0000
partition-geo,eyJpbmQiOjg3MzEsInRpbWVzdGFtcCI6IjIwMjAtMDctMTdUMDQ6Mzk6MDkiLCJsYXRpdHVkZSI6LTgzLjEwNCwibG9uZ2l0dWRlIjotMTcxLjMwMiwiY291bnRyeSI6IkFydWJhIn0=,streaming-0affec486183-geo,shardId-000000000000,49654776715983955342252074463363797440912524182737125378,2024-08-12T17:29:25.913+0000
partition-geo,eyJpbmQiOjEzMTMsInRpbWVzdGFtcCI6IjIwMTgtMDYtMjZUMDI6Mzk6MjUiLCJsYXRpdHVkZSI6NzcuMDQ0NywibG9uZ2l0dWRlIjo2MS45MTE5LCJjb3VudHJ5IjoiTWFsZGl2ZXMifQ==,streaming-0affec486183-geo,shardId-000000000000,49654776715983955342252074464474800269138368394292101122,2024-08-12T17:29:26.567+0000
partition-geo,eyJpbmQiOjQzMTUsInRpbWVzdGFtcCI6IjIwMTktMTItMTVUMDM6NTE6MjgiLCJsYXRpdHVkZSI6LTQ1Ljg1MDgsImxvbmdpdHVkZSI6NjYuMTAwMywiY291bnRyeSI6IkNvdGUgZCdJdm9pcmUifQ==,streaming-0affec486183-geo,shardId-000000000000,49654776715983955342252074466548108049777457635071623170,2024-08-12T17:29:28.198+0000
partition-geo,eyJpbmQiOjEwNzk0LCJ0aW1lc3RhbXAiOiIyMDIyLTAxLTAxVDAyOjI2OjUwIiwibGF0aXR1ZGUiOi04OS41MjM2LCJsb25naXR1ZGUiOi0xNTQuNTY3LCJjb3VudHJ5IjoiQ29jb3MgKEtlZWxpbmcpIElzbGFuZHMifQ==,streaming-0affec486183-geo,shardId-000000000000,49654776715983955342252074467375013310393864059290124290,2024-08-12T17:29:28.855+0000
partition-geo,eyJpbmQiOjU0OTQsInRpbWVzdGFtcCI6IjIwMjEtMDctMjFUMDI6MDI6MzUiLCJsYXRpdHVkZSI6LTgyLjY3NjgsImxvbmdpdHVkZSI6LTEyOS4yMDIsImNvdW50cnkiOiJCdWxnYXJpYSJ9,streaming-0affec486183-geo,shardId-000000000000,49654776715983955342252074468154770464045299876975607810,2024-08-12T17:29:29.506+0000
partition-geo,eyJpbmQiOjUwNjksInRpbWVzdGFtcCI6IjIwMjEtMDMtMjBUMDk6MzI6NDQiLCJsYXRpdHVkZSI6LTYzLjAwNjMsImxvbmdpdHVkZSI6LTE1Ny40NzQsImNvdW50cnkiOiJBemVyYmFpamFuIn0=,streaming-0affec486183-geo,shardId-000000000000,49654776715983955342252074468929691914418277246681743362,2024-08-12T17:29:30.158+0000


In [0]:
# need to be sending live data for this to work:
kstream_user = get_kinesis_stream('streaming-0affec486183-user')
display(kstream_user)

partitionKey,data,stream,shardId,sequenceNumber,approximateArrivalTimestamp


In [0]:
display(kstream_user)

partitionKey,data,stream,shardId,sequenceNumber,approximateArrivalTimestamp


In [0]:
df_pin = deserialize_kinesis_stream(kstream_pin, pin_schema)
df_geo = deserialize_kinesis_stream(kstream_geo, geo_schema)
df_user = deserialize_kinesis_stream(kstream_user, user_schema)

In [0]:
# replace empty entries and entries with no relevant data in each column with Nones
# column names and values to change to null
replace_dict = {
    "description": "No description available%",
    "follower_count": "User Info Error",
    "image_src": "Image src error.",
    "poster_name": "User Info Error",
    "tag_list": "N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e",
    "title": "No Title Data Available"
}
# replace wrong values with None
for key, value in replace_dict.items():
    df_pin = df_pin.withColumn(key, when(col(key).like(value), None).otherwise(col(key)))

# format follower_count as number
df_pin = df_pin.withColumn("follower_count", regexp_replace("follower_count", "k", "000"))
df_pin = df_pin.withColumn("follower_count", regexp_replace("follower_count", "M", "000000"))

# cast follower_count to integer type
df_pin = df_pin.withColumn("follower_count", col("follower_count").cast('int'))

# convert save_location column to include only the save location path
df_pin = df_pin.withColumn("save_location", regexp_replace("save_location", "Local save in ", ""))

# rename the index column to ind
df_pin = df_pin.withColumnRenamed("index", "ind")

# reorder the df_pin
new_order_pin = [
    "ind",
    "unique_id",
    "title",
    "description",
    "follower_count",
    "poster_name",
    "tag_list",
    "is_image_or_video",
    "image_src",
    "save_location",
    "category"
]
df_pin = df_pin.select(new_order_pin)
     

In [0]:
from pyspark.sql.types import ArrayType, DoubleType

# New coordinates column, array with lat, long
df_geo = df_geo.withColumn("coordinates", array("latitude", "longitude"))

# drop the lat and long cols
cols_to_drop = ("latitude", "longitude")
df_geo = df_geo.drop(*cols_to_drop)

# convert timestamp column to type timestamp
df_geo = df_geo.withColumn("timestamp", to_timestamp("timestamp"))

# reorder the df_geo
new_order_geo = [
    "ind",
    "country",
    "coordinates",
    "timestamp",
]
df_geo = df_geo.select(new_order_geo)
     

In [0]:
# create new user_name from first_name and last_name
df_user = df_user.withColumn("user_name", concat("first_name", "last_name"))

# drop first_name and last_name
df_user = df_user.drop('first_name', 'last_name')

# convert date_joined to timestamp
df_user = df_user.withColumn("date_joined", to_timestamp("date_joined"))

# reorder the df_user
new_order_user = [
    "ind",
    "user_name",
    "age",
    "date_joined",
]
df_user = df_user.select(new_order_user)

In [0]:
# insepct df_pin
display(df_pin)

ind,unique_id,title,description,follower_count,poster_name,tag_list,is_image_or_video,image_src,save_location,category
7528,fbe53c66-3442-4773-b19e-d3ec6f54dddf,null,null,null,null,null,multi-video(story page format),null,/data/mens-fashion,mens-fashion
2863,9bf39437-42a6-4f02-99a0-9a0383d8cd70,25 Super Fun Summer Crafts for Kids - Of Life and Lisa,Keep the kids busy this summer with these easy diy crafts and projects. Creative and…,124000,Of Life & Lisa | Lifestyle Blog,"Summer Crafts For Kids,Fun Crafts For Kids,Summer Kids,Toddler Crafts,Crafts To Do,Diy For Kids,Summer Snow,Diys For Summer,Craft Ideas For Girls",image,https://i.pinimg.com/originals/b3/bc/e2/b3bce2964e8c8975387b39660eed5f16.jpg,/data/diy-and-crafts,diy-and-crafts
5730,1e1f0c8b-9fcf-460b-9154-c775827206eb,Island Oasis Coupon Organizer,"Description Coupon Organizer in a fun colorful fabric -island oasis, Great Size for the ""basic"" couponer - holds up to 500 coupons with ease, and is made long enough so that you…",0,Consuelo Aguirre,"Grocery Items,Grocery Coupons,Care Organization,Coupon Organization,Extreme Couponing,Couponing 101,Life Binder,Save My Money,Love Coupons",image,https://i.pinimg.com/originals/65/bb/ea/65bbeaf458907bb079317d8303c4fa0e.jpg,/data/finance,finance
8304,5b6d0913-25e4-43ab-839d-85d5516f78a4,The #1 Reason You’re Not His Priority Anymore - Matthew Coast,#lovequotes #matchmaker #matchmadeinheaven #loveyourself #respectyourself,51000,Commitment Connection,"Wise Quotes,Quotable Quotes,Words Quotes,Wise Words,Quotes To Live By,Great Quotes,Motivational Quotes,Inspirational Quotes,Funny Quotes",image,https://i.pinimg.com/originals/c6/64/ee/c664ee71524fb5a6e7b7b49233f93b43.png,/data/quotes,quotes
8731,ea760f71-febf-4023-b592-d17396659039,20 Koi Fish Tattoos For Lucky Men,"Koi fish tattoos are a popular choice for men who want to make a statement, thanks to their rich symbolism and bold design.",211000,TheTrendSpotter,"Dr Tattoo,Wörter Tattoos,Pisces Tattoos,Tatoo Art,Dream Tattoos,Dope Tattoos,Mini Tattoos,Finger Tattoos,Body Art Tattoos",image,https://i.pinimg.com/originals/8a/0c/0a/8a0c0a7b6236565c519acd41ad1a52c0.jpg,/data/tattoos,tattoos
1313,44662045-e891-4821-8a19-ebe7eedd371a,Liquid Lash Extensions Mascara,"Instantly create the look of lash extensions with this award-winning, best-selling mascara that won't clump, flake or smudge. Available in 3 shades!",43000,Thrive Causemetics,null,video,https://i.pinimg.com/videos/thumbnails/originals/69/84/e2/6984e20f3e262098fa9c0614c3453254.0000001.jpg,/data/beauty,beauty
4315,21b59ba9-829d-4c33-8c27-4cd4c56d26b8,Podcasts for Teachers or Parents of Teenagers,"Podcasts for Teachers or Parents of Teenagers: Teaching teens middle school and high school can feel joyful and rewarding most days, but can also frustrate you with one challeng…",25000,Math Giraffe,"Middle School Classroom,High School Students,High School Teachers,Middle School Tips,High School Counseling,Ela Classroom,High School Science,Future Classroom,Google Classroom",image,https://i.pinimg.com/originals/50/19/31/501931a27ee4d076658980851b995b2c.jpg,/data/education,education
10794,c4bd2577-a7bb-4409-bb7a-17d5ed7e1cf1,TireBuyer,Nissan GT-R. Sick.,437,Ray Uyemura,"Lowrider,Old Vintage Cars,Antique Cars,Austin Martin,Nissan Gtr Black,Jaguar,1959 Cadillac,Cadillac Ct6,Old School Cars",image,https://i.pinimg.com/originals/0d/29/9f/0d299f3df020395aa7ce8387f40fbeed.jpg,/data/vehicles,vehicles
5494,8fb2af68-543b-4639-8119-de33d28706ed,Dave Ramsey's 7 Baby Steps: What Are They And Will They Work For You,"If you love budgeting, make sure to give Dave Ramsey's 7 Baby Steps a try. Follow these steps to begin your debt snowball, build an emergency fund, invest and reach riches. I ca…",26000,"Living Low Key | Save Money, Make Money, & Frugal Living","Financial Peace,Financial Tips,Saving Money Quotes,Total Money Makeover,Budgeting Finances,Money Management,Wealth Management,Personal Finance,Making Ideas",image,https://i.pinimg.com/originals/1e/9d/90

In [0]:
#df_pin.schema
#df_pin.schema.simpleString()
df_pin.printSchema()

root
-- ind: integer (nullable = true)
-- unique_id: string (nullable = true)
-- title: string (nullable = true)
-- description: string (nullable = true)
-- follower_count: integer (nullable = true)
-- poster_name: string (nullable = true)
-- tag_list: string (nullable = true)
-- is_image_or_video: string (nullable = true)
-- image_src: string (nullable = true)
-- save_location: string (nullable = true)
-- category: string (nullable = true)

In [0]:
display(df_geo)

ind,country,coordinates,timestamp
7528,Albania,"List(-89.9787, -173.293)",2020-08-28T03:52:47.000+0000
2863,Armenia,"List(-5.34445, -177.924)",2020-04-27T13:34:16.000+0000
5730,Colombia,"List(-77.015, -101.437)",2021-04-19T17:37:03.000+0000
8304,French Guiana,"List(-28.8852, -164.87)",2019-09-13T04:50:29.000+0000
8731,Aruba,"List(-83.104, -171.302)",2020-07-17T04:39:09.000+0000
1313,Maldives,"List(77.0447, 61.9119)",2018-06-26T02:39:25.000+0000
4315,Cote d'Ivoire,"List(-45.8508, 66.1003)",2019-12-15T03:51:28.000+0000
10794,Cocos (Keeling) Islands,"List(-89.5236, -154.567)",2022-01-01T02:26:50.000+0000
5494,Bulgaria,"List(-82.6768, -129.202)",2021-07-21T02:02:35.000+0000
5069,Azerbaijan,"List(-63.0063, -157.474)",2021-03-20T09:32:44.000+0000


In [0]:
df_geo.printSchema()

root
-- ind: integer (nullable = true)
-- country: string (nullable = true)
-- coordinates: array (nullable = false)
 |-- element: float (containsNull = true)
-- timestamp: timestamp (nullable = true)

In [0]:
display(df_user)

ind,user_name,age,date_joined


In [0]:
df_user.printSchema()

root
-- ind: integer (nullable = true)
-- user_name: string (nullable = true)
-- age: string (nullable = true)
-- date_joined: timestamp (nullable = true)

In [0]:
print((df_pin.count(), len(df.columns)))
# get this error: why?
# AnalysisException: Queries with streaming sources must be executed with writeStream.start();

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-1126979784802560> in <module> 
 ----> 1 print ( ( df_pin . count ( ) , len ( df . columns ) ) ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in count (self) 
 686 2 
 687 """
 --> 688 return int ( self . _jdf . count ( ) ) 
 689 
 690 def collect ( self ) : 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 AnalysisException : Queries with streaming sources must be executed with writeStream.start();
kinesis

In [0]:
write_Kinesis_stream_df_to_table(df_pin, "0affec486183", "pin")
write_Kinesis_stream_df_to_table(df_geo, "0affec486183", "geo")
write_Kinesis_stream_df_to_table(df_user, "0affec486183", "user")

## Some Stream Analytics

In [0]:
sliding_window_df_pin = (
    kstream_pin
    .groupBy(window("shardId", "1 hour", "30 minutes"))
    .agg(count("Data"))
)
#AnalysisException: Column 'timestamp' does not exist. Did you mean one of the following? [stream, data, shardId, partitionKey, sequenceNumber, approximateArrivalTimestamp];
print(sliding_window_df)

DataFrame[window: struct<start:timestamp,end:timestamp>, count(Data): bigint]

In [0]:
display(sliding_window_df_pin)

window,count(Data)


In [0]:
sliding_window_df_geo = (
    kstream_geo
    .groupBy(window("shardId", "1 hour", "30 minutes"))
    .agg(count("Data"))
)

In [0]:
display(sliding_window_df_geo)

window,count(Data)


In [0]:
sliding_window_df_user = (
    kstream_geo
    .groupBy(window("shardId", "1 hour", "30 minutes"))
    .agg(count("Data"))
)

In [0]:
display(sliding_window_df_user)

window,count(Data)
